# Modèles prédictifs du nombre de vélibs d'un point de vue spatial du 4 janvier 2021 au 12 février 2021

In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.express as px
import plotly.graph_objects as go


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import prettytable as ptbl

In [9]:
velib = pd.read_csv(r"/Users/Romain/Documents/Romain/ENSAE 2A 2020-2021/Stat'App/Github Stat'App/velib_nombre.csv")
velib = velib.drop(['Unnamed: 0'], axis = 1)
velib

,10/19/20;18:56:54,10/19/20;19:56:58,10/19/20;20:57:02,10/19/20;21:57:05,10/19/20;22:57:14,10/19/20;23:57:18,10/20/20;00:57:22,10/20/20;01:57:26,10/20/20;02:57:30,10/20/20;03:57:34,...,02/15/21;11:49:48,02/15/21;12:53:25,02/15/21;13:56:59,02/15/21;15:00:34,02/15/21;16:04:08,02/15/21;17:07:43,02/15/21;18:11:18,02/15/21;19:16:38,02/15/21;20:20:06,02/15/21;21:23:35
0,1,2,5,4,3,3,3,3,2,2,...,2,2,2,1,2,4,2,5,3,4
1,45,48,49,51,52,52,52,52,52,45,...,37,37,39,41,40,38,32,25,24,24
2,2,3,10,9,11,9,9,9,9,9,...,1,1,0,4,4,1,1,4,5,3
3,6,9,10,12,10,10,11,11,11,11,...,6,5,8,8,5,4,6,4,5,8
4,7,16,25,24,23,24,24,24,24,24,...,6,3,4,2,2,4,6,11,17,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,15,25,27,31,32,33,33,32,32,33,...,22,21,21,19,21,23,30,34,36,36
1394,2,2,0,14,16,16,16,16,16,16,...,2,3,3,4,5,6,8,9,9,11
1395,6,15,29,34,35,35,35,35,35,35,...,8,8,13,9,8,7,10,22,25,29
1396,17,15,12,15,12,12,12,12,12,12,...,11,11,16,14,14,14,15,7,9,8


On dispose d'une base de données nettoyées avec un échantillonnage par heure sur toutes les stations Vélib d'Ile-de-France. 

Avant de commencer les analyses, on garde toutefois le numéro des stations ainsi que leurs informations : 

In [10]:
# Import des librairies pour importer les données open data au format Json
import urllib.request
import json
from pandas.io.json import json_normalize

In [11]:
request= urllib.request.Request('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json')
response = urllib.request.urlopen(request)
station_info = response.read()
data = json.loads(station_info)
df_station_info = json_normalize(data['data']['stations'])
df_station_info.head()

,station_id,name,lat,lon,capacity,stationCode,rental_methods
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN
1,99950133,André Mazet - Saint-André des Arts,48.853756,2.339096,55,6015,[CREDITCARD]
2,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,NaN
3,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD]
4,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN


In [12]:
df_id_name_capacity = df_station_info[["station_id", "lat", "lon", "name", "capacity"]]
velib_details = df_id_name_capacity.merge(velib, how = "inner", left_index=True, right_index=True)
#velib_w_capacity = velib_w_capacity[velib_w_capacity["capacity"] != 0]

In [13]:
velib_details[:10]

,station_id,lat,lon,name,capacity,10/19/20;18:56:54,10/19/20;19:56:58,10/19/20;20:57:02,10/19/20;21:57:05,10/19/20;22:57:14,...,02/15/21;11:49:48,02/15/21;12:53:25,02/15/21;13:56:59,02/15/21;15:00:34,02/15/21;16:04:08,02/15/21;17:07:43,02/15/21;18:11:18,02/15/21;19:16:38,02/15/21;20:20:06,02/15/21;21:23:35
0,213688169,48.865983,2.275725,Benjamin Godard - Victor Hugo,35,1,2,5,4,3,...,2,2,2,1,2,4,2,5,3,4
1,99950133,48.853756,2.339096,André Mazet - Saint-André des Arts,55,45,48,49,51,52,...,37,37,39,41,40,38,32,25,24,24
2,516709288,48.855908,2.392571,Charonne - Robert et Sonia Delauney,20,2,3,10,9,11,...,1,1,0,4,4,1,1,4,5,3
3,36255,48.879296,2.337360,Toudouze - Clauzel,21,6,9,10,12,10,...,6,5,8,8,5,4,6,4,5,8
4,37815204,48.840855,2.387555,Mairie du 12ème,30,7,16,25,24,23,...,6,3,4,2,2,4,6,11,17,18
5,100769544,48.851519,2.343670,Harpe - Saint-Germain,46,38,26,18,20,24,...,26,25,29,26,27,27,28,28,28,29
6,85002689,48.819428,2.343335,Jourdan - Stade Charléty,60,19,23,19,29,30,...,6,11,11,10,10,9,11,17,18,20
7,54000559,48.881973,2.301132,Jouffroy d'Abbans - Wagram,40,9,8,5,7,7,...,13,13,15,10,10,10,17,17,15,15
8,85043758,48.882878,2.287667,Guersant - Gouvion-Saint-Cyr,29,4,10,11,14,14,...,29,27,29,28,30,31,35,37,38,38
9,123095125,48.871044,2.366104,Alibert - Jemmapes,60,13,25,37,36,36,...,25,22,18,13,14,13,21,26,28,33


In [14]:
df_velib = velib.transpose()
df_velib

,0,1,2,3,4,5,6,7,8,9,...,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397
10/19/20;18:56:54,1,45,2,6,7,38,19,9,4,13,...,6,4,2,38,18,15,2,6,17,17
10/19/20;19:56:58,2,48,3,9,16,26,23,8,10,25,...,7,6,1,21,20,25,2,15,15,13
10/19/20;20:57:02,5,49,10,10,25,18,19,5,11,37,...,5,12,1,18,17,27,0,29,12,5
10/19/20;21:57:05,4,51,9,12,24,20,29,7,14,36,...,6,14,2,17,19,31,14,34,15,6
10/19/20;22:57:14,3,52,11,10,23,24,30,7,14,36,...,6,13,0,18,19,32,16,35,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/15/21;17:07:43,4,38,1,4,4,27,9,10,31,13,...,3,10,2,15,11,23,6,7,14,22
02/15/21;18:11:18,2,32,1,6,6,28,11,17,35,21,...,5,11,2,11,11,30,8,10,15,11
02/15/21;19:16:38,5,25,4,4,11,28,17,17,37,26,...,11,16,3,11,12,34,9,22,7,6
02/15/21;20:20:06,3,24,5,5,17,28,18,15,38,28,...,10,18,1,13,12,36,9,25,9,5


In [15]:
def choix_dates(start,end,df):
    """
    start : jour du début des observations
    end : jour de fi des observations
    df : le dataframe des stations avec l'heure de pointage en index 

    Renvoie un sous dataframe indexé avec les dates qui nous intéressent
    """
    toutes_les_dates = df.index.tolist() #toutes les dates du dataframe
    dates_choisies = [] # contiendra le jour de début et de fin qui nous intéressent

    for index in df.index:
        if index.startswith(start):
            dates_choisies.append(index)
        elif index.startswith(end):
            dates_choisies.append(index)
    # On a donc une liste qui contient toutes les heures du jour de début et celles du jour de fin

    index_debut = toutes_les_dates.index(dates_choisies[0])
    index_fin = toutes_les_dates.index(dates_choisies[-1])
    liste_dates_choisies = toutes_les_dates[index_debut:index_fin]

    return(df.loc[liste_dates_choisies,:])

# On l'applique
df_velib_court = choix_dates('01/04/21', '02/12/21', df_velib)
df_velib_court

,0,1,2,3,4,5,6,7,8,9,...,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,12,16,5,21,10,33,3,26,10,20
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,13,16,5,19,10,33,4,27,10,20
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,13,16,5,19,10,22,4,27,10,20
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,13,17,5,19,10,9,4,15,10,20
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,13,17,5,19,10,9,4,15,10,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/12/21;18:13:07,4,27,2,15,9,32,12,6,23,41,...,5,15,4,19,13,9,7,21,19,5
02/12/21;19:16:34,4,26,5,10,15,24,19,10,26,39,...,9,18,2,16,12,8,7,25,19,6
02/12/21;20:20:05,4,23,2,11,13,25,19,9,28,41,...,9,18,1,17,13,13,7,26,16,4
02/12/21;21:23:31,4,24,3,12,12,25,17,12,29,43,...,12,20,0,19,13,15,8,26,16,4


## Construction de la variable à prédire : le nombre de vélos dans une station target **1 jour** plus tard

In [16]:
df = df_velib_court.copy()
df['y'] = df[0]
df

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,y
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,7
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,7
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,7
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,7
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/12/21;18:13:07,4,27,2,15,9,32,12,6,23,41,...,15,4,19,13,9,7,21,19,5,4
02/12/21;19:16:34,4,26,5,10,15,24,19,10,26,39,...,18,2,16,12,8,7,25,19,6,4
02/12/21;20:20:05,4,23,2,11,13,25,19,9,28,41,...,18,1,17,13,13,7,26,16,4,4
02/12/21;21:23:31,4,24,3,12,12,25,17,12,29,43,...,20,0,19,13,15,8,26,16,4,4


In [17]:
axis = df_velib_court.index

# Boucle qui met dans Y toutes les données 24h plus tard
for i in range(len(df)-24):
    df['y'][axis[i]] = df[0][axis[i+24]]

# Puis on supprime les 24 dernières lignes qui ne peuvent pas être complétées
df.drop(axis[-24:], inplace = True)
df

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,y
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,10
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,10
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,10
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,10
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/11/21;16:50:25,8,20,3,5,17,45,9,23,18,47,...,6,1,4,16,5,7,11,14,26,4
02/11/21;17:53:56,5,22,4,8,22,40,10,21,22,47,...,11,1,5,18,8,8,9,19,30,4
02/11/21;18:57:19,8,18,7,8,29,42,12,19,23,44,...,11,1,2,18,7,9,12,20,21,4
02/11/21;20:00:43,3,17,5,9,30,42,20,14,27,49,...,18,0,7,18,13,9,16,20,24,4


## Machine Learning basique : regarder ce qu'il se passe sur une station

In [18]:
# Choix des features
stations = [i for i in df.columns if i != 'y']
X = df[stations]

# Préparation de la target
y = df['y']

In [19]:
# Régression linéaire simple
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_prediction = regressor.predict(X_test)

print("Le MAE du modèle est "+str(round(mean_absolute_error(y_test, y_prediction),3))+" avec la régression linéaire")

Le MAE du modèle est 5.06 avec la régression linéaire


In [22]:
# Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
regressor = RandomForestRegressor(max_depth = 5, min_samples_split = 4, n_estimators = 50)
regressor.fit(X_train, y_train)
y_prediction = regressor.predict(X_test)

print("Le MAE du modèle est "+str(round(mean_absolute_error(y_test, y_prediction),3))+" avec random forest")

Le MAE du modèle est 4.015 avec random forest


## Réalisation d'un apprentissage pour 10 stations aléatoires (les premières dans l'ordre des données) : automatisation

In [23]:
# Import du module de visualisation des résultats
from prettytable import PrettyTable

In [24]:
# Liste des stations cibles
stations_tests = []
for i in range(10):
    stations_tests.append("station_"+str(i))
stations_tests

['station_0',
 'station_1',
 'station_2',
 'station_3',
 'station_4',
 'station_5',
 'station_6',
 'station_7',
 'station_8',
 'station_9']

## Visualisation des données sélectionnées

In [25]:
import folium

# Titre : 
loc = 'Les 10 stations Velib selectionnees'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

stations_velib_paris = folium.Map(location = [48.856578, 2.351828], zoom_start = 12.5) # On centre sur Paris
for i in range(10): # Affichage des 10 premières stations considérées
    
    test_mark  = '<head><meta http-equiv="Content-Type" content="text/html; charset=windows-1252"></head>'
    test_mark += '<strong>'+stations_tests[i] + " " + df_station_info['name'][i] + ". Capacite = "+str(velib_details['capacity'][i])+'<strong>'

    folium.Marker(
        np.array([df_station_info['lat'][i],df_station_info['lon'][i]]).tolist(),
        #popup = stations_tests[i] + " " + df_station_info['name'][i] + ". Capacite = "+str(velib_details['capacity'][i]), 
        popup = test_mark, 
        icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(stations_velib_paris)

stations_velib_paris.get_root().html.add_child(folium.Element(title_html))

stations_velib_paris

## Machine Learning

In [47]:
# Stockage des index du dataframe
axis = df_velib_court.index
# Préparation des résultats à stocker : pour chaque station, on garde le numéro de la station dans le dataframe df_velib, le modèle de 
# régression linéaire et de random forest
resultats = {}

# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - test", "Std(Occupation) - test", "MAE LR - test", "MAE RF - test", "MAE méthode intuitive - test"]

# Dataframe pour la Régression Linéaire et le Random Forest
for i in range(len(stations_tests)):
    df = df_velib_court.copy()
    df['y'] = df[i]

    for j in range(len(df)-24):
        df['y'][axis[j]] = df[i][axis[j+24]]
        
    # Puis on supprime les 24 dernières lignes qui ne peuvent pas être complétées
    df.drop(axis[-24:], inplace = True)

    # Machine Learning
    stations_features = [i for i in df.columns if i != 'y']
    X = df[stations_features]
    # Préparation de la target
    y = df['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[i], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[i].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[i].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[i])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats[stations_tests[i]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[i], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)
print("Résultats des MAE sur le test")

+----------------+-------------------------+------------------------+---------------+---------------+------------------------------+
| Station target | Mean(Occupation) - test | Std(Occupation) - test | MAE LR - test | MAE RF - test | MAE méthode intuitive - test |
+----------------+-------------------------+------------------------+---------------+---------------+------------------------------+
|   station_0    |           5.07          |          3.69          |      5.06     |      3.95     |             3.98             |
|   station_1    |          35.32          |         10.49          |     18.27     |      9.47     |             6.9              |
|   station_2    |           5.12          |          3.41          |      4.25     |      2.44     |             3.53             |
|   station_3    |           6.47          |          3.56          |      5.99     |      2.62     |             3.94             |
|   station_4    |          16.21          |          8.42          |

On s'intéresse à présent à la moyenne des erreurs de prédiction des modèles : la moyenne des MAE

In [48]:
ptbl1 = PrettyTable()
#ptbl.field_names = ["Occupation moyenne des stations cibles", "Moyenne MAE LR", "Moyenne MAE RF", "Moyenne MAE méthode naïve"]
ptbl1.field_names = ["Mean(Occupation) - test", "Std(Occupation) - test","Mean(MAE) LR - 1 modèle - test", "Mean(MAE) RF - 1 modèle - test", "Mean(MAE) naïf - 1 modèle - test"]

mean_occupation1 = 0
mean_std_occupation1 = 0
mean_MAE_lr = 0
mean_MAE_rf = 0
mean_MAE_naif = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats.items():
    mean_occupation1 += resultats[key][9]
    mean_std_occupation1 += resultats[key][15]
    mean_MAE_lr += resultats[key][4]
    mean_MAE_rf += resultats[key][6]
    mean_MAE_naif += resultats[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_occupation1 = round(mean_occupation1/len(resultats),2)
mean_std_occupation1 = round(mean_std_occupation1/len(resultats),2)
mean_MAE_lr = round(mean_MAE_lr/len(resultats),2)
mean_MAE_rf = round(mean_MAE_rf/len(resultats),2)
mean_MAE_naif = round(mean_MAE_naif/len(resultats),2)

# Visualisation des performances avec Prettytable
ptbl1.add_row([mean_occupation1, mean_std_occupation1, mean_MAE_lr, mean_MAE_rf, mean_MAE_naif])
print(ptbl1)

+-------------------------+------------------------+--------------------------------+--------------------------------+----------------------------------+
| Mean(Occupation) - test | Std(Occupation) - test | Mean(MAE) LR - 1 modèle - test | Mean(MAE) RF - 1 modèle - test | Mean(MAE) naïf - 1 modèle - test |
+-------------------------+------------------------+--------------------------------+--------------------------------+----------------------------------+
|          17.29          |          7.75          |              9.24              |              6.67              |               6.28               |
+-------------------------+------------------------+--------------------------------+--------------------------------+----------------------------------+


On regarde ensuite à quelle heure de la journée est-ce que nous faisons le plus d'erreurs de prédiction.

## Moyenne des erreurs par heure

On rappelle que, pour la station i, 
- `resultats[stations_tests[i]][10]` contient `y_test` du ML et colonne a le nom `y`
- `resultats[stations_tests[i]][11]` contient les prédictions de `y_test` avec LR et colonne a le nom `0`
- `resultats[stations_tests[i]][12]` contient les prédictions de `y_test` avec LR et colonne a le nom `0`
- `resultats[stations_tests[i]][13]` contient `y_test` de la méthode naïve et colonne a le nom `i`
- `resultats[stations_tests[i]][14]` contient les prédictions de `y_test` avec méthode naïve et colonne a le nom `y`

Fonction qui indique l'heure du pointage dans une nouvelle colonne `heure`

In [49]:
def get_heure(df):
    df['heure'] = df.index 
    for i in df.index:
        df['heure'][i] = float(df['heure'][i][9]+df['heure'][i][10])

On applique à présent cette fonction sur tous les dataframes des tests :

In [50]:
for i in stations_tests:
    get_heure(resultats[i][10])
    get_heure(resultats[i][11])
    get_heure(resultats[i][12])
    get_heure(resultats[i][13])
    get_heure(resultats[i][14])

On crée maintenant le dataframe de chaque station qui contient les erreurs de prédiction à chaque date en valeur absolue

In [51]:
for i in range(len(stations_tests)):
    # Initialisation
    resultats[stations_tests[i]][10]['erreur_pred_lr'] = 0
    resultats[stations_tests[i]][10]['erreur_pred_rf'] = 0
    resultats[stations_tests[i]][10]['erreur_pred_naif'] = 0
    # On stocke les erreurs en valeur absolue
    resultats[stations_tests[i]][10]['erreur_pred_lr'] = abs(resultats[stations_tests[i]][11][0] - resultats[stations_tests[i]][10]['y'])
    resultats[stations_tests[i]][10]['erreur_pred_rf'] = abs(resultats[stations_tests[i]][12][0] - resultats[stations_tests[i]][10]['y'])
    resultats[stations_tests[i]][10]['erreur_pred_naif'] = abs(resultats[stations_tests[i]][13][i] - resultats[stations_tests[i]][14]['y'])

On crée à présent le dataframe de toutes les erreurs de prédiction suivant les différents modèles sur les 10 stations

In [52]:
# Initialisation
df_pred_errors = resultats[stations_tests[0]][10]

for i in stations_tests[1:]:
    df_pred_errors = pd.concat([df_pred_errors, resultats[i][10]])

df_pred_errors.head(5)

,y,heure,erreur_pred_lr,erreur_pred_rf,erreur_pred_naif
01/31/21;03:40:41,3,3,1.111694,0.446251,6
01/31/21;04:43:42,3,4,2.542588,0.363754,6
01/31/21;05:46:46,3,5,3.032736,0.422328,6
01/31/21;06:49:50,2,6,4.545158,1.603620,7
01/31/21;07:52:53,3,7,2.927713,0.704693,7


In [53]:
mean_erreur_par_heure = round(df_pred_errors.groupby('heure').mean(),2)
mean_erreur_par_heure

,y,erreur_pred_lr,erreur_pred_rf,erreur_pred_naif
heure,,,,
0.0,19.67,9.77,7.12,6.34
1.0,19.84,10.09,7.32,6.65
2.0,19.52,9.93,7.02,6.56
3.0,19.33,9.08,6.51,6.39
4.0,19.26,9.32,6.55,6.31
5.0,19.20,9.34,6.65,6.29
6.0,16.94,9.45,6.23,6.26
7.0,16.88,10.04,6.71,6.91
8.0,15.48,10.35,6.90,6.82


In [54]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_lr'],
    name="Erreur moyenne avec Régression Linéaire"      
))


fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_rf'],
    name="Erreur moyenne avec Forêt aléatoire"     
))

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_naif'],
    name="Erreur moyenne avec la méthode naïve"     
))


fig.update_layout(
    title="Moyenne des erreurs de prédiction par heure sur 10 stations",
    xaxis_title="Heure",
    yaxis_title="Erreur moyenne",
    legend_title="",
    autosize=False,
    width=1000,
    height=500
)

fig.show()

In [264]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_lr'],
    name="Erreur moyenne avec Régression Linéaire"      
))


fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_rf'],
    name="Erreur moyenne avec Forêt aléatoire"     
))

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_naif'],
    name="Erreur moyenne avec la méthode naïve"     
))


# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[4,4, 12.5,12.5],
    y=[11,10.5, 11, 10.5],
    text=["Nuit", "19h-5h59",
          "Journée", "6h-18h59"],
    mode="text",
    showlegend = False

))

# Set axes ranges
fig.update_xaxes(range=[0, 23])
fig.update_yaxes(range=[5, 12.5])

# Add shapes
fig.add_shape(type="line",
    x0=6, y0=0, x1=6, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=19, y0=0, x1=19, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)




fig.update_layout(
    title="Erreurs de prédictions sur les 10 stations du 04/01/21 au 12/02/21",
    xaxis_title="Heure",
    yaxis_title="Erreur moyenne",
    legend_title="",
    autosize=False,
    width=1000,
    height=500
)

fig.show()

# Tuning des hyperparamètres du Random Forest : essai avec la 1ère station

In [35]:
df = df_velib_court.copy()
df['y'] = df[0]
df

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,y
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,7
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,7
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,7
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,7
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/12/21;18:13:07,4,27,2,15,9,32,12,6,23,41,...,15,4,19,13,9,7,21,19,5,4
02/12/21;19:16:34,4,26,5,10,15,24,19,10,26,39,...,18,2,16,12,8,7,25,19,6,4
02/12/21;20:20:05,4,23,2,11,13,25,19,9,28,41,...,18,1,17,13,13,7,26,16,4,4
02/12/21;21:23:31,4,24,3,12,12,25,17,12,29,43,...,20,0,19,13,15,8,26,16,4,4


In [36]:
axis = df_velib_court.index

# Boucle qui met dans Y toutes les données 24h plus tard
for i in range(len(df)-24):
    df['y'][axis[i]] = df[0][axis[i+24]]

# Puis on supprime les 24 dernières lignes qui ne peuvent pas être complétées
df.drop(axis[-24:], inplace = True)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,y
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,10
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,10
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,10
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,10
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,10


In [37]:
# Choix des features
stations = [i for i in df.columns if i != 'y']
X = df[stations]

# Préparation de la target
y = df['y']

# Séparation en données de test et d'entrainement
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)

## Random Search Cross Validation

In [38]:
rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mae',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


### Random Hyperparameter Grid

In [39]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


### Random Search training

In [40]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train[:100], y_train[:100])


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning:

tostring() is deprecated. Use tobytes() instead.

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  9.4min finished
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

On regarde les mailleurs paramètres avec la recherche aléatoire

In [41]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60,
 'bootstrap': False}

### GridSearch

In [ ]:
{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60,
 'bootstrap': False}

In [42]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [30, 40, 70, 100],
    #'max_features': [2, 3],
    'min_samples_split': [7, 9, 10, 12],
    'n_estimators': [100, 200, 400, 600]
}

# Create a base model
rf = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2, return_train_score=True)

In [43]:
# Fit the grid search to the data
grid_search.fit(X_train[:100], y_train[:100])

Fitting 10 folds for each of 64 candidates, totalling 640 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning:

tostring() is deprecated. Use tobytes() instead.

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed: 40.6min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'bootstrap': [False], 'max_depth': [30, 40, 70, 100], 'min_samples_split': [7, 9, 10, 12], 'n_estimators': [100, 200, 400, 600]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [44]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 30,
 'min_samples_split': 7,
 'n_estimators': 600}

**Quelle amélioration ?**

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
rf_simple = RandomForestRegressor()
rf_boost = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
rf_simple.fit(X_train, y_train)
rf_boost.fit(X_train, y_train)

y_prediction_simple = rf_simple.predict(X_test)
y_prediction_boost = rf_boost.predict(X_test)

MAE_simple = mean_absolute_error(y_test, y_prediction_simple)
MAE_boost = mean_absolute_error(y_test, y_prediction_boost)

print("Le MAE du modèle simple est "+str(round(MAE_simple,3))+" avec random forest")
print("Le MAE du modèle boosté est "+str(round(MAE_boost,3))+" avec random forest")

Le MAE du modèle simple est 4.408 avec random forest
Le MAE du modèle boosté est 3.844 avec random forest


**Soit un gain de ...**

In [77]:
print("L'optimisation des paramètres du Random Forest a conduit à une variation de "+str(round((MAE_boost - MAE_simple) / MAE_simple *100,2))+"%"+" du MAE")

L'optimisation des paramètres du Random Forest a conduit à une variation de -12.8% du MAE


## Machine Learning sur les périodes choisies

In [120]:
df_velib_court.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,12,16,5,21,10,33,3,26,10,20
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,13,16,5,19,10,33,4,27,10,20
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,13,16,5,19,10,22,4,27,10,20
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,13,17,5,19,10,9,4,15,10,20
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,13,17,5,19,10,9,4,15,10,20


In [121]:
df_h = df_velib_court.copy()
df_h['heure'] = df_velib_court.index
df_h.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,heure
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,01/04/21;00:03:28
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,01/04/21;01:05:43
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,01/04/21;02:07:51
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,01/04/21;03:10:01
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,01/04/21;04:12:06


In [122]:
# On ne garde que les heures dans la date
for i in range(len(df_h)):
    df_h['heure'][i] = int(df_h['heure'][i][9]+df_h['heure'][i][10])

df_h.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,heure
01/04/21;00:03:28,7,25,5,5,21,37,3,20,20,17,...,16,5,21,10,33,3,26,10,20,0
01/04/21;01:05:43,7,25,5,5,21,37,3,20,19,17,...,16,5,19,10,33,4,27,10,20,1
01/04/21;02:07:51,7,25,5,5,21,37,3,20,19,18,...,16,5,19,10,22,4,27,10,20,2
01/04/21;03:10:01,7,25,5,5,22,24,3,20,19,18,...,17,5,19,10,9,4,15,10,20,3
01/04/21;04:12:06,7,25,5,6,22,25,16,20,19,18,...,17,5,19,10,9,4,15,10,20,4


# Modèles prédictifs la journée (6h00-18h59) heure par heure

In [123]:
heures = np.linspace(0,23,24)
heures_jour = np.linspace(6,18,13)

In [139]:
def sous_df_1h(df, h):
    """
    Renvoie le sous dataframe avec l'heure target

    h : l'heure target
    df : le dataframe court sur la période choisie par Malo
    """
    pas_les_bonnes_heures = list(np.linspace(0,23,24))
    pas_les_bonnes_heures.remove(h)
    df_c = df.copy()
    test = df_c.isin({'heure': pas_les_bonnes_heures})

    for i in df_c.index:
        if test['heure'][i] == True:
            df_c.drop(i,0,inplace = True)

    # Et on supprime la colonne des heures
    del df_c['heure']

    return df_c

In [134]:
def sous_df_ml(df_1h, num_station):
    """
    Renvoie le dataframe avec la variable target 'y' : PRÉPARATION AU ML

    num_station : le numéro de la station cible
    df : le sous_df
    """
    axis = df_1h.index
    df_c = df_1h.copy()
    df_c['y'] = df_c[num_station]

    for j in range(len(df_c)-24):
        df_c['y'][axis[j]] = df_c[num_station][axis[j+24]]

    # Puis on supprime les 24 dernières lignes qui ne peuvent pas être complétées
    df_c.drop(axis[-24:], inplace = True)

    return df_c

# Résultats 0h

In [200]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_0h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 0h", "Std(Occupation) - 0h", "MAE LR - 0h", "MAE RF - 0h", "MAE méthode intuitive - 0h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 0h
    df_1h = sous_df_1h(df_h,0)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_0h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 0h | Std(Occupation) - 0h | MAE LR - 0h | MAE RF - 0h | MAE méthode intuitive - 0h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          5.2          |         3.03         |     5.35    |     3.93    |            5.2             |
|   station_1    |          42.2         |         5.72         |    16.37    |    17.06    |            18.8            |
|   station_2    |          3.8          |         3.11         |     2.23    |     1.24    |            1.6             |
|   station_3    |          6.4          |         4.88         |     2.79    |     2.68    |            5.8             |
|   station_4    |          24.2         |         5.89         |     5.53    |     5.14    |            6.6             |
|   station_5   

In [201]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 0h", "Mean(Std(Occupation)) / 0h", "Moyenne MAE LR / 0h", "Moyenne MAE RF / 0h", "Moyenne MAE méthode naïve / 0h"]
mean_capacity_0h = 0
mean_std_occupation_0h = 0
mean_MAE_lr_0h = 0
mean_MAE_rf_0h = 0
mean_MAE_naif_0h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_0h.items():
    mean_capacity_0h += resultats_0h[key][9]
    mean_std_occupation_0h += resultats_0h[key][15]
    mean_MAE_lr_0h += resultats_0h[key][4]
    mean_MAE_rf_0h += resultats_0h[key][6]
    mean_MAE_naif_0h += resultats_0h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_0h = round(mean_capacity_0h/len(resultats_0h),2)
mean_std_occupation_0h = round(mean_std_occupation_0h/len(resultats_0h),2)
mean_MAE_lr_0h = round(mean_MAE_lr_0h/len(resultats_0h),2)
mean_MAE_rf_0h = round(mean_MAE_rf_0h/len(resultats_0h),2)
mean_MAE_naif_0h = round(mean_MAE_naif_0h/len(resultats_0h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_0h, mean_std_occupation_0h, mean_MAE_lr_0h, mean_MAE_rf_0h, mean_MAE_naif_0h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 0h | Mean(Std(Occupation)) / 0h | Moyenne MAE LR / 0h | Moyenne MAE RF / 0h | Moyenne MAE méthode naïve / 0h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         18.62         |            4.54            |         7.74        |         7.34        |              9.26              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats 1h

In [202]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_1h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 1h", "Std(Occupation) - 1h", "MAE LR - 1h", "MAE RF - 1h", "MAE méthode intuitive - 1h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 1h
    df_1h = sous_df_1h(df_h,1)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_1h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 1h | Std(Occupation) - 1h | MAE LR - 1h | MAE RF - 1h | MAE méthode intuitive - 1h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          5.2          |         2.59         |     4.54    |     4.35    |            5.2             |
|   station_1    |          42.4         |         5.18         |    17.65    |    15.32    |            18.8            |
|   station_2    |          4.0          |         3.08         |     2.35    |     1.16    |            1.4             |
|   station_3    |          7.2          |         4.38         |     2.71    |     3.18    |            6.0             |
|   station_4    |          23.8         |         5.59         |     4.27    |     4.79    |            6.2             |
|   station_5   

In [203]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 1h", "Mean(Std(Occupation)) / 1h", "Moyenne MAE LR / 1h", "Moyenne MAE RF / 1h", "Moyenne MAE méthode naïve / 1h"]
mean_capacity_1h = 0
mean_std_occupation_1h = 0
mean_MAE_lr_1h = 0
mean_MAE_rf_1h = 0
mean_MAE_naif_1h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_1h.items():
    mean_capacity_1h += resultats_1h[key][9]
    mean_std_occupation_1h += resultats_1h[key][15]
    mean_MAE_lr_1h += resultats_1h[key][4]
    mean_MAE_rf_1h += resultats_1h[key][6]
    mean_MAE_naif_1h += resultats_1h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_1h = round(mean_capacity_1h/len(resultats_1h),2)
mean_std_occupation_1h = round(mean_std_occupation_1h/len(resultats_1h),2)
mean_MAE_lr_1h = round(mean_MAE_lr_1h/len(resultats_1h),2)
mean_MAE_rf_1h = round(mean_MAE_rf_1h/len(resultats_1h),2)
mean_MAE_naif_1h = round(mean_MAE_naif_1h/len(resultats_1h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_1h, mean_std_occupation_1h, mean_MAE_lr_1h, mean_MAE_rf_1h, mean_MAE_naif_1h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 1h | Mean(Std(Occupation)) / 1h | Moyenne MAE LR / 1h | Moyenne MAE RF / 1h | Moyenne MAE méthode naïve / 1h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         19.02         |            4.37            |         7.77        |         7.05        |              9.24              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats 2h

In [204]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_2h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 2h", "Std(Occupation) - 2h", "MAE LR - 2h", "MAE RF - 2h", "MAE méthode intuitive - 2h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 2h
    df_1h = sous_df_1h(df_h,2)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_2h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 2h | Std(Occupation) - 2h | MAE LR - 2h | MAE RF - 2h | MAE méthode intuitive - 2h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          6.2          |         2.17         |     3.76    |     4.1     |            4.6             |
|   station_1    |          33.4         |         16.1         |    17.04    |    14.17    |            13.6            |
|   station_2    |          4.4          |         2.07         |     3.68    |     2.85    |            1.0             |
|   station_3    |          5.4          |         4.28         |     4.5     |     4.4     |            5.2             |
|   station_4    |          20.6         |         7.54         |     5.56    |     4.84    |            5.2             |
|   station_5   

In [205]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 2h", "Mean(Std(Occupation)) / 2h", "Moyenne MAE LR / 2h", "Moyenne MAE RF / 2h", "Moyenne MAE méthode naïve / 2h"]
mean_capacity_2h = 0
mean_std_occupation_2h = 0
mean_MAE_lr_2h = 0
mean_MAE_rf_2h = 0
mean_MAE_naif_2h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_2h.items():
    mean_capacity_2h += resultats_2h[key][9]
    mean_std_occupation_2h += resultats_2h[key][15]
    mean_MAE_lr_2h += resultats_2h[key][4]
    mean_MAE_rf_2h += resultats_2h[key][6]
    mean_MAE_naif_2h += resultats_2h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_2h = round(mean_capacity_2h/len(resultats_2h),2)
mean_std_occupation_2h = round(mean_std_occupation_2h/len(resultats_2h),2)
mean_MAE_lr_2h = round(mean_MAE_lr_2h/len(resultats_2h),2)
mean_MAE_rf_2h = round(mean_MAE_rf_2h/len(resultats_2h),2)
mean_MAE_naif_2h = round(mean_MAE_naif_2h/len(resultats_2h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_2h, mean_std_occupation_2h, mean_MAE_lr_2h, mean_MAE_rf_2h, mean_MAE_naif_2h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 2h | Mean(Std(Occupation)) / 2h | Moyenne MAE LR / 2h | Moyenne MAE RF / 2h | Moyenne MAE méthode naïve / 2h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         17.62         |            5.9             |         7.76        |         7.06        |              8.32              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats 3h

In [206]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_3h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 3h", "Std(Occupation) - 3h", "MAE LR - 3h", "MAE RF - 3h", "MAE méthode intuitive - 3h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 3h
    df_1h = sous_df_1h(df_h,3)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_3h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 3h | Std(Occupation) - 3h | MAE LR - 3h | MAE RF - 3h | MAE méthode intuitive - 3h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          4.75         |         1.71         |     4.79    |     4.66    |            4.25            |
|   station_1    |          33.0         |        11.66         |    20.51    |    19.58    |            18.0            |
|   station_2    |          4.25         |         1.89         |     3.52    |     2.98    |            4.0             |
|   station_3    |          9.5          |         1.0          |     4.35    |     4.25    |            3.75            |
|   station_4    |          19.5         |         5.45         |     5.99    |     7.69    |            8.75            |
|   station_5   

In [207]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 3h", "Mean(Std(Occupation)) / 3h", "Moyenne MAE LR / 3h", "Moyenne MAE RF / 3h", "Moyenne MAE méthode naïve / 3h"]
mean_capacity_3h = 0
mean_std_occupation_3h = 0
mean_MAE_lr_3h = 0
mean_MAE_rf_3h = 0
mean_MAE_naif_3h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_3h.items():
    mean_capacity_3h += resultats_3h[key][9]
    mean_std_occupation_3h += resultats_3h[key][15]
    mean_MAE_lr_3h += resultats_3h[key][4]
    mean_MAE_rf_3h += resultats_3h[key][6]
    mean_MAE_naif_3h += resultats_3h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_3h = round(mean_capacity_3h/len(resultats_3h),2)
mean_std_occupation_3h = round(mean_std_occupation_3h/len(resultats_3h),2)
mean_MAE_lr_3h = round(mean_MAE_lr_3h/len(resultats_3h),2)
mean_MAE_rf_3h = round(mean_MAE_rf_3h/len(resultats_3h),2)
mean_MAE_naif_3h = round(mean_MAE_naif_3h/len(resultats_3h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_3h, mean_std_occupation_3h, mean_MAE_lr_3h, mean_MAE_rf_3h, mean_MAE_naif_3h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 3h | Mean(Std(Occupation)) / 3h | Moyenne MAE LR / 3h | Moyenne MAE RF / 3h | Moyenne MAE méthode naïve / 3h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         17.45         |            5.15            |         9.22        |         8.97        |              9.82              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats 4h

In [208]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_4h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 4h", "Std(Occupation) - 4h", "MAE LR - 4h", "MAE RF - 4h", "MAE méthode intuitive - 4h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 4h
    df_1h = sous_df_1h(df_h,4)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_4h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 4h | Std(Occupation) - 4h | MAE LR - 4h | MAE RF - 4h | MAE méthode intuitive - 4h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          4.5          |         2.65         |     4.37    |     4.48    |            6.0             |
|   station_1    |          32.5         |        16.13         |    15.83    |    17.49    |           16.25            |
|   station_2    |          4.25         |         2.5          |     3.82    |     3.53    |            2.75            |
|   station_3    |          7.5          |         3.87         |     3.91    |     4.21    |            4.25            |
|   station_4    |         24.25         |         2.5          |     6.14    |     7.0     |            3.0             |
|   station_5   

In [209]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 4h", "Mean(Std(Occupation)) / 4h", "Moyenne MAE LR / 4h", "Moyenne MAE RF / 4h", "Moyenne MAE méthode naïve / 4h"]
mean_capacity_4h = 0
mean_std_occupation_4h = 0
mean_MAE_lr_4h = 0
mean_MAE_rf_4h = 0
mean_MAE_naif_4h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_4h.items():
    mean_capacity_4h += resultats_4h[key][9]
    mean_std_occupation_4h += resultats_4h[key][15]
    mean_MAE_lr_4h += resultats_4h[key][4]
    mean_MAE_rf_4h += resultats_4h[key][6]
    mean_MAE_naif_4h += resultats_4h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_4h = round(mean_capacity_4h/len(resultats_4h),2)
mean_std_occupation_4h = round(mean_std_occupation_4h/len(resultats_4h),2)
mean_MAE_lr_4h = round(mean_MAE_lr_4h/len(resultats_4h),2)
mean_MAE_rf_4h = round(mean_MAE_rf_4h/len(resultats_4h),2)
mean_MAE_naif_4h = round(mean_MAE_naif_4h/len(resultats_4h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_4h, mean_std_occupation_4h, mean_MAE_lr_4h, mean_MAE_rf_4h, mean_MAE_naif_4h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 4h | Mean(Std(Occupation)) / 4h | Moyenne MAE LR / 4h | Moyenne MAE RF / 4h | Moyenne MAE méthode naïve / 4h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         17.43         |            5.35            |         8.72        |         8.55        |              8.95              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats 5h

In [210]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_5h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 5h", "Std(Occupation) - 5h", "MAE LR - 5h", "MAE RF - 5h", "MAE méthode intuitive - 5h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 5h
    df_1h = sous_df_1h(df_h,5)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_5h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 5h | Std(Occupation) - 5h | MAE LR - 5h | MAE RF - 5h | MAE méthode intuitive - 5h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          3.75         |         2.22         |     5.06    |     4.33    |            4.0             |
|   station_1    |         40.75         |         7.59         |    16.52    |     15.1    |            20.0            |
|   station_2    |          3.5          |         2.52         |     3.32    |     3.61    |            2.75            |
|   station_3    |          8.0          |         3.56         |     4.7     |     4.56    |            3.5             |
|   station_4    |          21.5         |         6.95         |     7.75    |     5.75    |            7.5             |
|   station_5   

In [211]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 5h", "Mean(Std(Occupation)) / 5h", "Moyenne MAE LR / 5h", "Moyenne MAE RF / 5h", "Moyenne MAE méthode naïve / 5h"]
mean_capacity_5h = 0
mean_std_occupation_5h = 0
mean_MAE_lr_5h = 0
mean_MAE_rf_5h = 0
mean_MAE_naif_5h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_5h.items():
    mean_capacity_5h += resultats_5h[key][9]
    mean_std_occupation_5h += resultats_5h[key][15]
    mean_MAE_lr_5h += resultats_5h[key][4]
    mean_MAE_rf_5h += resultats_5h[key][6]
    mean_MAE_naif_5h += resultats_5h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_5h = round(mean_capacity_5h/len(resultats_5h),2)
mean_std_occupation_5h = round(mean_std_occupation_5h/len(resultats_5h),2)
mean_MAE_lr_5h = round(mean_MAE_lr_5h/len(resultats_5h),2)
mean_MAE_rf_5h = round(mean_MAE_rf_5h/len(resultats_5h),2)
mean_MAE_naif_5h = round(mean_MAE_naif_5h/len(resultats_5h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_5h, mean_std_occupation_5h, mean_MAE_lr_5h, mean_MAE_rf_5h, mean_MAE_naif_5h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 5h | Mean(Std(Occupation)) / 5h | Moyenne MAE LR / 5h | Moyenne MAE RF / 5h | Moyenne MAE méthode naïve / 5h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         17.85         |            4.49            |         9.09        |         7.94        |             10.42              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats à 6h

In [212]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_6h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 6h", "Std(Occupation) - 6h", "MAE LR - 6h", "MAE RF - 6h", "MAE méthode intuitive - 6h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 6h
    df_1h = sous_df_1h(df_h,6)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_6h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 6h | Std(Occupation) - 6h | MAE LR - 6h | MAE RF - 6h | MAE méthode intuitive - 6h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          3.75         |         2.63         |     4.52    |     4.59    |            2.25            |
|   station_1    |          39.0         |         4.69         |     6.99    |     7.07    |           11.25            |
|   station_2    |          4.0          |         2.58         |     3.93    |     3.84    |            4.0             |
|   station_3    |          9.5          |         1.29         |     4.57    |     4.54    |            4.25            |
|   station_4    |          21.0         |         6.38         |     3.35    |     5.16    |            9.75            |
|   station_5   

In [213]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 6h", "Mean(Std(Occupation)) / 6h", "Moyenne MAE LR / 6h", "Moyenne MAE RF / 6h", "Moyenne MAE méthode naïve / 6h"]
mean_capacity_6h = 0
mean_std_occupation_6h = 0
mean_MAE_lr_6h = 0
mean_MAE_rf_6h = 0
mean_MAE_naif_6h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_6h.items():
    mean_capacity_6h += resultats_6h[key][9]
    mean_std_occupation_6h += resultats_6h[key][15]
    mean_MAE_lr_6h += resultats_6h[key][4]
    mean_MAE_rf_6h += resultats_6h[key][6]
    mean_MAE_naif_6h += resultats_6h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_6h = round(mean_capacity_6h/len(resultats_6h),2)
mean_std_occupation_6h = round(mean_std_occupation_6h/len(resultats_6h),2)
mean_MAE_lr_6h = round(mean_MAE_lr_6h/len(resultats_6h),2)
mean_MAE_rf_6h = round(mean_MAE_rf_6h/len(resultats_6h),2)
mean_MAE_naif_6h = round(mean_MAE_naif_6h/len(resultats_6h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_6h, mean_std_occupation_6h, mean_MAE_lr_6h, mean_MAE_rf_6h, mean_MAE_naif_6h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 6h | Mean(Std(Occupation)) / 6h | Moyenne MAE LR / 6h | Moyenne MAE RF / 6h | Moyenne MAE méthode naïve / 6h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         18.23         |            4.3             |         5.79        |         5.66        |              8.88              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats à 7h

In [214]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_7h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 7h", "Std(Occupation) - 7h", "MAE LR - 7h", "MAE RF - 7h", "MAE méthode intuitive - 7h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 7h
    df_1h = sous_df_1h(df_h,7)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_7h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 7h | Std(Occupation) - 7h | MAE LR - 7h | MAE RF - 7h | MAE méthode intuitive - 7h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          5.6          |         1.52         |     4.03    |     4.01    |            4.8             |
|   station_1    |          32.2         |        18.19         |    14.94    |    11.29    |            14.8            |
|   station_2    |          3.8          |         1.64         |     2.42    |     2.3     |            2.6             |
|   station_3    |          5.2          |         3.27         |     3.96    |     4.13    |            4.0             |
|   station_4    |          19.8         |         7.63         |     5.32    |     6.03    |            6.4             |
|   station_5   

In [215]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 7h", "Mean(Std(Occupation)) / 7h", "Moyenne MAE LR / 7h", "Moyenne MAE RF / 7h", "Moyenne MAE méthode naïve / 7h"]
mean_capacity_7h = 0
mean_std_occupation_7h = 0
mean_MAE_lr_7h = 0
mean_MAE_rf_7h = 0
mean_MAE_naif_7h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_7h.items():
    mean_capacity_7h += resultats_7h[key][9]
    mean_std_occupation_7h += resultats_7h[key][15]
    mean_MAE_lr_7h += resultats_7h[key][4]
    mean_MAE_rf_7h += resultats_7h[key][6]
    mean_MAE_naif_7h += resultats_7h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_7h = round(mean_capacity_7h/len(resultats_7h),2)
mean_std_occupation_7h = round(mean_std_occupation_7h/len(resultats_7h),2)
mean_MAE_lr_7h = round(mean_MAE_lr_7h/len(resultats_7h),2)
mean_MAE_rf_7h = round(mean_MAE_rf_7h/len(resultats_7h),2)
mean_MAE_naif_7h = round(mean_MAE_naif_7h/len(resultats_7h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_7h, mean_std_occupation_7h, mean_MAE_lr_7h, mean_MAE_rf_7h, mean_MAE_naif_7h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 7h | Mean(Std(Occupation)) / 7h | Moyenne MAE LR / 7h | Moyenne MAE RF / 7h | Moyenne MAE méthode naïve / 7h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|          17.6         |            6.09            |         7.11        |         6.73        |              8.54              |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats à 8h

In [216]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_8h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 8h", "Std(Occupation) - 8h", "MAE LR - 8h", "MAE RF - 8h", "MAE méthode intuitive - 8h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 8h
    df_1h = sous_df_1h(df_h,8)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_8h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 8h | Std(Occupation) - 8h | MAE LR - 8h | MAE RF - 8h | MAE méthode intuitive - 8h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          4.2          |         2.77         |     4.0     |     3.38    |            2.8             |
|   station_1    |          38.2         |        10.31         |    18.18    |     15.1    |            17.6            |
|   station_2    |          4.0          |         1.58         |     2.34    |     1.91    |            3.2             |
|   station_3    |          4.6          |         2.88         |     3.16    |     3.45    |            3.8             |
|   station_4    |          15.6         |        10.01         |     6.98    |     7.61    |            4.4             |
|   station_5   

In [217]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 8h", "Mean(Std(Occupation)) / 9h", "Moyenne MAE LR / 8h", "Moyenne MAE RF / 8h", "Moyenne MAE méthode naïve / 8h"]
mean_capacity_8h = 0
mean_std_occupation_8h = 0
mean_MAE_lr_8h = 0
mean_MAE_rf_8h = 0
mean_MAE_naif_8h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_8h.items():
    mean_capacity_8h += resultats_8h[key][9]
    mean_std_occupation_8h += resultats_8h[key][15]
    mean_MAE_lr_8h += resultats_8h[key][4]
    mean_MAE_rf_8h += resultats_8h[key][6]
    mean_MAE_naif_8h += resultats_8h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_8h = round(mean_capacity_8h/len(resultats_8h),2)
mean_std_occupation_8h = round(mean_std_occupation_8h/len(resultats_8h),2)
mean_MAE_lr_8h = round(mean_MAE_lr_8h/len(resultats_8h),2)
mean_MAE_rf_8h = round(mean_MAE_rf_8h/len(resultats_8h),2)
mean_MAE_naif_8h = round(mean_MAE_naif_8h/len(resultats_8h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_8h, mean_std_occupation_8h, mean_MAE_lr_8h, mean_MAE_rf_8h, mean_MAE_naif_8h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 8h | Mean(Std(Occupation)) / 9h | Moyenne MAE LR / 8h | Moyenne MAE RF / 8h | Moyenne MAE méthode naïve / 8h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         16.44         |            5.57            |         8.47        |         7.69        |              8.7               |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats à 9h

In [218]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_9h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 9h", "Std(Occupation) - 9h", "MAE LR - 9h", "MAE RF - 9h", "MAE méthode intuitive - 9h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 9h
    df_1h = sous_df_1h(df_h,9)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_9h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
| Station target | Mean(Occupation) - 9h | Std(Occupation) - 9h | MAE LR - 9h | MAE RF - 9h | MAE méthode intuitive - 9h |
+----------------+-----------------------+----------------------+-------------+-------------+----------------------------+
|   station_0    |          4.4          |         1.52         |     3.96    |     3.65    |            3.4             |
|   station_1    |          35.2         |        15.12         |    14.19    |     9.53    |            13.2            |
|   station_2    |          4.2          |         3.11         |     3.59    |     3.96    |            3.2             |
|   station_3    |          1.6          |         1.52         |     2.07    |     1.56    |            2.2             |
|   station_4    |          14.8         |        11.56         |     6.53    |     6.74    |            9.8             |
|   station_5   

In [219]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 9h", "Mean(Std(Occupation)) / 9h", "Moyenne MAE LR / 9h", "Moyenne MAE RF / 9h", "Moyenne MAE méthode naïve / 9h"]
mean_capacity_9h = 0
mean_std_occupation_9h = 0
mean_MAE_lr_9h = 0
mean_MAE_rf_9h = 0
mean_MAE_naif_9h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_9h.items():
    mean_capacity_9h += resultats_9h[key][9]
    mean_std_occupation_9h += resultats_9h[key][15]
    mean_MAE_lr_9h += resultats_9h[key][4]
    mean_MAE_rf_9h += resultats_9h[key][6]
    mean_MAE_naif_9h += resultats_9h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_9h = round(mean_capacity_9h/len(resultats_9h),2)
mean_std_occupation_9h = round(mean_std_occupation_9h/len(resultats_9h),2)
mean_MAE_lr_9h = round(mean_MAE_lr_9h/len(resultats_9h),2)
mean_MAE_rf_9h = round(mean_MAE_rf_9h/len(resultats_9h),2)
mean_MAE_naif_9h = round(mean_MAE_naif_9h/len(resultats_9h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_9h, mean_std_occupation_9h, mean_MAE_lr_9h, mean_MAE_rf_9h, mean_MAE_naif_9h])
print(ptbl)

+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
| Mean(Occupation) / 9h | Mean(Std(Occupation)) / 9h | Moyenne MAE LR / 9h | Moyenne MAE RF / 9h | Moyenne MAE méthode naïve / 9h |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+
|         15.38         |            6.24            |         7.91        |         7.21        |              9.5               |
+-----------------------+----------------------------+---------------------+---------------------+--------------------------------+


# Résultats à 10h

In [220]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_10h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 10h", "Std(Occupation) - 10h", "MAE LR - 10h", "MAE RF - 10h", "MAE méthode intuitive - 10h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 10h
    df_1h = sous_df_1h(df_h,10)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_10h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 10h | Std(Occupation) - 10h | MAE LR - 10h | MAE RF - 10h | MAE méthode intuitive - 10h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          3.25          |          0.5          |     4.31     |     3.01     |             3.75            |
|   station_1    |          40.5          |          5.32         |    16.26     |    14.42     |            21.25            |
|   station_2    |          3.5           |          3.7          |     3.61     |     3.33     |             5.25            |
|   station_3    |          4.25          |          2.22         |     2.59     |     2.97     |             4.75            |
|   station_4    |         10.75          |          12.2         |     7.21     |     6.77     |       

In [221]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 10h", "Mean(Std(Occupation)) / 10h", "Moyenne MAE LR / 10h", "Moyenne MAE RF / 10h", "Moyenne MAE méthode naïve / 10h"]
mean_capacity_10h = 0
mean_std_occupation_10h = 0
mean_MAE_lr_10h = 0
mean_MAE_rf_10h = 0
mean_MAE_naif_10h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_10h.items():
    mean_capacity_10h += resultats_10h[key][9]
    mean_std_occupation_10h += resultats_10h[key][15]
    mean_MAE_lr_10h += resultats_10h[key][4]
    mean_MAE_rf_10h += resultats_10h[key][6]
    mean_MAE_naif_10h += resultats_10h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_10h = round(mean_capacity_10h/len(resultats_10h),2)
mean_std_occupation_10h = round(mean_std_occupation_10h/len(resultats_10h),2)
mean_MAE_lr_10h = round(mean_MAE_lr_10h/len(resultats_10h),2)
mean_MAE_rf_10h = round(mean_MAE_rf_10h/len(resultats_10h),2)
mean_MAE_naif_10h = round(mean_MAE_naif_10h/len(resultats_10h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_10h, mean_std_occupation_10h, mean_MAE_lr_10h, mean_MAE_rf_10h, mean_MAE_naif_10h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 10h | Mean(Std(Occupation)) / 10h | Moyenne MAE LR / 10h | Moyenne MAE RF / 10h | Moyenne MAE méthode naïve / 10h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|          14.5          |             4.7             |        10.08         |         9.05         |              11.98              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 11h

In [222]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_11h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 11h", "Std(Occupation) - 11h", "MAE LR - 11h", "MAE RF - 11h", "MAE méthode intuitive - 11h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 11h
    df_1h = sous_df_1h(df_h,11)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_11h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 11h | Std(Occupation) - 11h | MAE LR - 11h | MAE RF - 11h | MAE méthode intuitive - 11h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          3.4           |          1.14         |     4.12     |     4.38     |             3.6             |
|   station_1    |          38.0          |         11.77         |    14.48     |     12.4     |             18.0            |
|   station_2    |          3.0           |          2.65         |     2.35     |     1.89     |             3.8             |
|   station_3    |          4.8           |          4.66         |     3.14     |     3.38     |             3.8             |
|   station_4    |          13.2          |         12.91         |     5.68     |     6.45     |       

In [223]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 11h", "Mean(Std(Occupation)) / 11h", "Moyenne MAE LR / 11h", "Moyenne MAE RF / 11h", "Moyenne MAE méthode naïve / 11h"]
mean_capacity_11h = 0
mean_std_occupation_11h = 0
mean_MAE_lr_11h = 0
mean_MAE_rf_11h = 0
mean_MAE_naif_11h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_11h.items():
    mean_capacity_11h += resultats_11h[key][9]
    mean_std_occupation_11h += resultats_11h[key][15]
    mean_MAE_lr_11h += resultats_11h[key][4]
    mean_MAE_rf_11h += resultats_11h[key][6]
    mean_MAE_naif_11h += resultats_11h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_11h = round(mean_capacity_11h/len(resultats_11h),2)
mean_std_occupation_11h = round(mean_std_occupation_11h/len(resultats_11h),2)
mean_MAE_lr_11h = round(mean_MAE_lr_11h/len(resultats_11h),2)
mean_MAE_rf_11h = round(mean_MAE_rf_11h/len(resultats_11h),2)
mean_MAE_naif_11h = round(mean_MAE_naif_11h/len(resultats_11h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_11h, mean_std_occupation_11h, mean_MAE_lr_11h, mean_MAE_rf_11h, mean_MAE_naif_11h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 11h | Mean(Std(Occupation)) / 11h | Moyenne MAE LR / 11h | Moyenne MAE RF / 11h | Moyenne MAE méthode naïve / 11h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         14.52          |             5.71            |         8.68         |         8.47         |               9.12              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 12h

In [224]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_12h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 12h", "Std(Occupation) - 12h", "MAE LR - 12h", "MAE RF - 12h", "MAE méthode intuitive - 12h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 12h
    df_1h = sous_df_1h(df_h,12)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_12h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 12h | Std(Occupation) - 12h | MAE LR - 12h | MAE RF - 12h | MAE méthode intuitive - 12h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          1.8           |          1.1          |     2.42     |     1.88     |             3.6             |
|   station_1    |          39.4          |         11.87         |    19.15     |    18.08     |             19.6            |
|   station_2    |          2.4           |          2.07         |     3.33     |     2.91     |             3.4             |
|   station_3    |          3.8           |          4.97         |     3.76     |     4.04     |             4.2             |
|   station_4    |          13.6          |         10.53         |     5.91     |     5.7      |       

In [225]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 12h", "Mean(Std(Occupation)) / 12h", "Moyenne MAE LR / 12h", "Moyenne MAE RF / 12h", "Moyenne MAE méthode naïve / 12h"]
mean_capacity_12h = 0
mean_std_occupation_12h = 0
mean_MAE_lr_12h = 0
mean_MAE_rf_12h = 0
mean_MAE_naif_12h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_12h.items():
    mean_capacity_12h += resultats_12h[key][9]
    mean_std_occupation_12h += resultats_12h[key][15]
    mean_MAE_lr_12h += resultats_12h[key][4]
    mean_MAE_rf_12h += resultats_12h[key][6]
    mean_MAE_naif_12h += resultats_12h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_12h = round(mean_capacity_12h/len(resultats_12h),2)
mean_std_occupation_12h = round(mean_std_occupation_12h/len(resultats_12h),2)
mean_MAE_lr_12h = round(mean_MAE_lr_12h/len(resultats_12h),2)
mean_MAE_rf_12h = round(mean_MAE_rf_12h/len(resultats_12h),2)
mean_MAE_naif_12h = round(mean_MAE_naif_12h/len(resultats_12h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_12h, mean_std_occupation_12h, mean_MAE_lr_12h, mean_MAE_rf_12h, mean_MAE_naif_12h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 12h | Mean(Std(Occupation)) / 12h | Moyenne MAE LR / 12h | Moyenne MAE RF / 12h | Moyenne MAE méthode naïve / 12h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         13.84          |             6.01            |         8.59         |         7.96         |               8.28              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 13h

In [226]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_13h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 13h", "Std(Occupation) - 13h", "MAE LR - 13h", "MAE RF - 13h", "MAE méthode intuitive - 13h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 13h
    df_1h = sous_df_1h(df_h,13)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_13h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 13h | Std(Occupation) - 13h | MAE LR - 13h | MAE RF - 13h | MAE méthode intuitive - 13h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          3.6           |          2.3          |     2.52     |     2.12     |             4.0             |
|   station_1    |          38.6          |         10.74         |    18.33     |     14.3     |             17.4            |
|   station_2    |          2.8           |          1.64         |     2.35     |     1.89     |             2.4             |
|   station_3    |          2.4           |          2.88         |     2.6      |     2.56     |             3.0             |
|   station_4    |          12.0          |         10.54         |     5.9      |     6.65     |       

In [227]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 13h", "Mean(Std(Occupation)) / 13h", "Moyenne MAE LR / 13h", "Moyenne MAE RF / 13h", "Moyenne MAE méthode naïve / 13h"]
mean_capacity_13h = 0
mean_std_occupation_13h = 0
mean_MAE_lr_13h = 0
mean_MAE_rf_13h = 0
mean_MAE_naif_13h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_13h.items():
    mean_capacity_13h += resultats_13h[key][9]
    mean_std_occupation_13h += resultats_13h[key][15]
    mean_MAE_lr_13h += resultats_13h[key][4]
    mean_MAE_rf_13h += resultats_13h[key][6]
    mean_MAE_naif_13h += resultats_13h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_13h = round(mean_capacity_13h/len(resultats_13h),2)
mean_std_occupation_13h = round(mean_std_occupation_13h/len(resultats_13h),2)
mean_MAE_lr_13h = round(mean_MAE_lr_13h/len(resultats_13h),2)
mean_MAE_rf_13h = round(mean_MAE_rf_13h/len(resultats_13h),2)
mean_MAE_naif_13h = round(mean_MAE_naif_13h/len(resultats_13h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_13h, mean_std_occupation_13h, mean_MAE_lr_13h, mean_MAE_rf_13h, mean_MAE_naif_13h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 13h | Mean(Std(Occupation)) / 13h | Moyenne MAE LR / 13h | Moyenne MAE RF / 13h | Moyenne MAE méthode naïve / 13h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         13.64          |             6.01            |         8.22         |         7.08         |               8.14              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 14h

In [228]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_14h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 14h", "Std(Occupation) - 14h", "MAE LR - 14h", "MAE RF - 14h", "MAE méthode intuitive - 14h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 14h
    df_1h = sous_df_1h(df_h,14)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_14h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 14h | Std(Occupation) - 14h | MAE LR - 14h | MAE RF - 14h | MAE méthode intuitive - 14h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          4.0           |          1.58         |     2.7      |     2.57     |             3.8             |
|   station_1    |          45.8          |          5.85         |     15.9     |    13.25     |             19.2            |
|   station_2    |          2.8           |          2.05         |     1.81     |     1.34     |             1.0             |
|   station_3    |          3.0           |          2.55         |     2.24     |     2.34     |             3.2             |
|   station_4    |          12.4          |         11.63         |     6.91     |     6.55     |       

In [229]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 14h", "Mean(Std(Occupation)) / 14h", "Moyenne MAE LR / 14h", "Moyenne MAE RF / 14h", "Moyenne MAE méthode naïve / 14h"]
mean_capacity_14h = 0
mean_std_occupation_14h = 0
mean_MAE_lr_14h = 0
mean_MAE_rf_14h = 0
mean_MAE_naif_14h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_14h.items():
    mean_capacity_14h += resultats_14h[key][9]
    mean_std_occupation_14h += resultats_14h[key][15]
    mean_MAE_lr_14h += resultats_14h[key][4]
    mean_MAE_rf_14h += resultats_14h[key][6]
    mean_MAE_naif_14h += resultats_14h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_14h = round(mean_capacity_14h/len(resultats_14h),2)
mean_std_occupation_14h = round(mean_std_occupation_14h/len(resultats_14h),2)
mean_MAE_lr_14h = round(mean_MAE_lr_14h/len(resultats_14h),2)
mean_MAE_rf_14h = round(mean_MAE_rf_14h/len(resultats_14h),2)
mean_MAE_naif_14h = round(mean_MAE_naif_14h/len(resultats_14h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_14h, mean_std_occupation_14h, mean_MAE_lr_14h, mean_MAE_rf_14h, mean_MAE_naif_14h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 14h | Mean(Std(Occupation)) / 14h | Moyenne MAE LR / 14h | Moyenne MAE RF / 14h | Moyenne MAE méthode naïve / 14h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         15.18          |             4.72            |         6.32         |         6.24         |               8.56              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 15h

In [230]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_15h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 15h", "Std(Occupation) - 15h", "MAE LR - 15h", "MAE RF - 15h", "MAE méthode intuitive - 15h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 15h
    df_1h = sous_df_1h(df_h,15)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_15h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 15h | Std(Occupation) - 15h | MAE LR - 15h | MAE RF - 15h | MAE méthode intuitive - 15h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          2.4           |          1.95         |     3.16     |     3.23     |             2.8             |
|   station_1    |          38.8          |         14.96         |    18.51     |    15.89     |             17.0            |
|   station_2    |          2.4           |          2.07         |     3.12     |     2.08     |             2.2             |
|   station_3    |          2.0           |          2.0          |     2.28     |     1.98     |             2.8             |
|   station_4    |          14.8          |         11.45         |     9.5      |     9.64     |       

In [231]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 15h", "Mean(Std(Occupation)) / 15h", "Moyenne MAE LR / 15h", "Moyenne MAE RF / 15h", "Moyenne MAE méthode naïve / 15h"]
mean_capacity_15h = 0
mean_std_occupation_15h = 0
mean_MAE_lr_15h = 0
mean_MAE_rf_15h = 0
mean_MAE_naif_15h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_15h.items():
    mean_capacity_15h += resultats_15h[key][9]
    mean_std_occupation_15h += resultats_15h[key][15]
    mean_MAE_lr_15h += resultats_15h[key][4]
    mean_MAE_rf_15h += resultats_15h[key][6]
    mean_MAE_naif_15h += resultats_15h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_15h = round(mean_capacity_15h/len(resultats_15h),2)
mean_std_occupation_15h = round(mean_std_occupation_15h/len(resultats_15h),2)
mean_MAE_lr_15h = round(mean_MAE_lr_15h/len(resultats_15h),2)
mean_MAE_rf_15h = round(mean_MAE_rf_15h/len(resultats_15h),2)
mean_MAE_naif_15h = round(mean_MAE_naif_15h/len(resultats_15h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_15h, mean_std_occupation_15h, mean_MAE_lr_15h, mean_MAE_rf_15h, mean_MAE_naif_15h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 15h | Mean(Std(Occupation)) / 15h | Moyenne MAE LR / 15h | Moyenne MAE RF / 15h | Moyenne MAE méthode naïve / 15h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         13.46          |             6.71            |         8.26         |         7.25         |               9.54              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 16h

In [232]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_16h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 16h", "Std(Occupation) - 16h", "MAE LR - 16h", "MAE RF - 16h", "MAE méthode intuitive - 16h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 16h
    df_1h = sous_df_1h(df_h,16)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_16h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 16h | Std(Occupation) - 16h | MAE LR - 16h | MAE RF - 16h | MAE méthode intuitive - 16h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          4.25          |          1.89         |     3.77     |     4.0      |             3.25            |
|   station_1    |          46.0          |          4.9          |    23.24     |     23.1     |            25.25            |
|   station_2    |          2.0           |          1.41         |     3.19     |     2.08     |             1.75            |
|   station_3    |          3.0           |          1.41         |     1.53     |     1.73     |             3.25            |
|   station_4    |         12.75          |         12.53         |     9.42     |     9.81     |       

In [233]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 16h", "Mean(Std(Occupation)) / 16h", "Moyenne MAE LR / 16h", "Moyenne MAE RF / 16h", "Moyenne MAE méthode naïve / 16h"]
mean_capacity_16h = 0
mean_std_occupation_16h = 0
mean_MAE_lr_16h = 0
mean_MAE_rf_16h = 0
mean_MAE_naif_16h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_16h.items():
    mean_capacity_16h += resultats_16h[key][9]
    mean_std_occupation_16h += resultats_16h[key][15]
    mean_MAE_lr_16h += resultats_16h[key][4]
    mean_MAE_rf_16h += resultats_16h[key][6]
    mean_MAE_naif_16h += resultats_16h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_16h = round(mean_capacity_16h/len(resultats_16h),2)
mean_std_occupation_16h = round(mean_std_occupation_16h/len(resultats_16h),2)
mean_MAE_lr_16h = round(mean_MAE_lr_16h/len(resultats_16h),2)
mean_MAE_rf_16h = round(mean_MAE_rf_16h/len(resultats_16h),2)
mean_MAE_naif_16h = round(mean_MAE_naif_16h/len(resultats_16h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_16h, mean_std_occupation_16h, mean_MAE_lr_16h, mean_MAE_rf_16h, mean_MAE_naif_16h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 16h | Mean(Std(Occupation)) / 16h | Moyenne MAE LR / 16h | Moyenne MAE RF / 16h | Moyenne MAE méthode naïve / 16h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         14.28          |             4.02            |         9.98         |         8.91         |               10.3              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 17h

In [234]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_17h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 17h", "Std(Occupation) - 17h", "MAE LR - 17h", "MAE RF - 17h", "MAE méthode intuitive - 17h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 17h
    df_1h = sous_df_1h(df_h,17)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_17h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 17h | Std(Occupation) - 17h | MAE LR - 17h | MAE RF - 17h | MAE méthode intuitive - 17h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          4.2           |          2.59         |     2.82     |     2.96     |             3.6             |
|   station_1    |          44.6          |          12.1         |    20.18     |    18.88     |             21.4            |
|   station_2    |          3.2           |          2.17         |     3.76     |     3.29     |             2.6             |
|   station_3    |          2.0           |          0.71         |     3.02     |     3.25     |             3.6             |
|   station_4    |          14.6          |         10.38         |     8.91     |     9.2      |       

In [235]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 17h", "Mean(Std(Occupation)) / 17h", "Moyenne MAE LR / 17h", "Moyenne MAE RF / 17h", "Moyenne MAE méthode naïve / 17h"]
mean_capacity_17h = 0
mean_std_occupation_17h = 0
mean_MAE_lr_17h = 0
mean_MAE_rf_17h = 0
mean_MAE_naif_17h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_17h.items():
    mean_capacity_17h += resultats_17h[key][9]
    mean_std_occupation_17h += resultats_17h[key][15]
    mean_MAE_lr_17h += resultats_17h[key][4]
    mean_MAE_rf_17h += resultats_17h[key][6]
    mean_MAE_naif_17h += resultats_17h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_17h = round(mean_capacity_17h/len(resultats_17h),2)
mean_std_occupation_17h = round(mean_std_occupation_17h/len(resultats_17h),2)
mean_MAE_lr_17h = round(mean_MAE_lr_17h/len(resultats_17h),2)
mean_MAE_rf_17h = round(mean_MAE_rf_17h/len(resultats_17h),2)
mean_MAE_naif_17h = round(mean_MAE_naif_17h/len(resultats_17h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_17h, mean_std_occupation_17h, mean_MAE_lr_17h, mean_MAE_rf_17h, mean_MAE_naif_17h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 17h | Mean(Std(Occupation)) / 17h | Moyenne MAE LR / 17h | Moyenne MAE RF / 17h | Moyenne MAE méthode naïve / 17h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         14.92          |             5.18            |         8.74         |         7.87         |               8.42              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 18h

In [251]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_18h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 18h", "Std(Occupation) - 18h", "MAE LR - 18h", "MAE RF - 18h", "MAE méthode intuitive - 18h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 18h
    df_1h = sous_df_1h(df_h,18)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_18h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 18h | Std(Occupation) - 18h | MAE LR - 18h | MAE RF - 18h | MAE méthode intuitive - 18h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          6.0           |          2.83         |     3.73     |     2.96     |             3.0             |
|   station_1    |          41.6          |         15.09         |    16.34     |    14.35     |             18.2            |
|   station_2    |          4.2           |          2.17         |     1.52     |     1.49     |             2.8             |
|   station_3    |          2.4           |          2.3          |     4.61     |     4.86     |             5.4             |
|   station_4    |          24.2          |          5.5          |     8.8      |     9.06     |       

In [252]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 18h", "Mean(Std(Occupation)) / 18h", "Moyenne MAE LR / 18h", "Moyenne MAE RF / 18h", "Moyenne MAE méthode naïve / 18h"]
mean_capacity_18h = 0
mean_std_occupation_18h = 0
mean_MAE_lr_18h = 0
mean_MAE_rf_18h = 0
mean_MAE_naif_18h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_18h.items():
    mean_capacity_18h += resultats_18h[key][9]
    mean_std_occupation_18h += resultats_18h[key][15]
    mean_MAE_lr_18h += resultats_18h[key][4]
    mean_MAE_rf_18h += resultats_18h[key][6]
    mean_MAE_naif_18h += resultats_18h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_18h = round(mean_capacity_18h/len(resultats_18h),2)
mean_std_occupation_18h = round(mean_std_occupation_18h/len(resultats_18h),2)
mean_MAE_lr_18h = round(mean_MAE_lr_18h/len(resultats_18h),2)
mean_MAE_rf_18h = round(mean_MAE_rf_18h/len(resultats_18h),2)
mean_MAE_naif_18h = round(mean_MAE_naif_18h/len(resultats_18h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_18h, mean_std_occupation_18h, mean_MAE_lr_18h, mean_MAE_rf_18h, mean_MAE_naif_18h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 18h | Mean(Std(Occupation)) / 18h | Moyenne MAE LR / 18h | Moyenne MAE RF / 18h | Moyenne MAE méthode naïve / 18h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         17.18          |             5.71            |         8.51         |         7.48         |               8.66              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 19h

In [238]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_19h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 19h", "Std(Occupation) - 19h", "MAE LR - 19h", "MAE RF - 19h", "MAE méthode intuitive - 19h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 19h
    df_1h = sous_df_1h(df_h,19)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_19h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 19h | Std(Occupation) - 19h | MAE LR - 19h | MAE RF - 19h | MAE méthode intuitive - 19h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          4.5           |          1.29         |     3.42     |     2.5      |             2.5             |
|   station_1    |         41.25          |          3.59         |    13.24     |    14.11     |            13.75            |
|   station_2    |          2.0           |          1.83         |     1.35     |     1.25     |             2.25            |
|   station_3    |          4.75          |          1.89         |     4.2      |     4.05     |             4.5             |
|   station_4    |          16.5          |          3.32         |     3.45     |     3.0      |       

In [239]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 19h", "Mean(Std(Occupation)) / 19h", "Moyenne MAE LR / 19h", "Moyenne MAE RF / 19h", "Moyenne MAE méthode naïve / 19h"]
mean_capacity_19h = 0
mean_std_occupation_19h = 0
mean_MAE_lr_19h = 0
mean_MAE_rf_19h = 0
mean_MAE_naif_19h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_19h.items():
    mean_capacity_19h += resultats_19h[key][9]
    mean_std_occupation_19h += resultats_19h[key][15]
    mean_MAE_lr_19h += resultats_19h[key][4]
    mean_MAE_rf_19h += resultats_19h[key][6]
    mean_MAE_naif_19h += resultats_19h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_19h = round(mean_capacity_19h/len(resultats_19h),2)
mean_std_occupation_19h = round(mean_std_occupation_19h/len(resultats_19h),2)
mean_MAE_lr_19h = round(mean_MAE_lr_19h/len(resultats_19h),2)
mean_MAE_rf_19h = round(mean_MAE_rf_19h/len(resultats_19h),2)
mean_MAE_naif_19h = round(mean_MAE_naif_19h/len(resultats_19h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_19h, mean_std_occupation_19h, mean_MAE_lr_19h, mean_MAE_rf_19h, mean_MAE_naif_19h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 19h | Mean(Std(Occupation)) / 19h | Moyenne MAE LR / 19h | Moyenne MAE RF / 19h | Moyenne MAE méthode naïve / 19h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|          14.5          |             3.47            |         5.52         |         5.42         |               8.48              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 20h

In [240]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_20h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 20h", "Std(Occupation) - 20h", "MAE LR - 20h", "MAE RF - 20h", "MAE méthode intuitive - 20h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 20h
    df_1h = sous_df_1h(df_h,20)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_20h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 20h | Std(Occupation) - 20h | MAE LR - 20h | MAE RF - 20h | MAE méthode intuitive - 20h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          6.4           |          3.13         |     5.04     |     4.64     |             5.6             |
|   station_1    |          36.4          |         18.66         |    19.87     |     17.5     |             19.2            |
|   station_2    |          3.6           |          2.3          |     1.21     |     1.09     |             2.0             |
|   station_3    |          4.6           |          2.97         |     2.47     |     2.48     |             5.2             |
|   station_4    |          26.6          |          3.21         |     9.14     |     8.98     |       

In [241]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 20h", "Mean(Std(Occupation)) / 20h", "Moyenne MAE LR / 20h", "Moyenne MAE RF / 20h", "Moyenne MAE méthode naïve / 20h"]
mean_capacity_20h = 0
mean_std_occupation_20h = 0
mean_MAE_lr_20h = 0
mean_MAE_rf_20h = 0
mean_MAE_naif_20h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_20h.items():
    mean_capacity_20h += resultats_20h[key][9]
    mean_std_occupation_20h += resultats_20h[key][15]
    mean_MAE_lr_20h += resultats_20h[key][4]
    mean_MAE_rf_20h += resultats_20h[key][6]
    mean_MAE_naif_20h += resultats_20h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_20h = round(mean_capacity_20h/len(resultats_20h),2)
mean_std_occupation_20h = round(mean_std_occupation_20h/len(resultats_20h),2)
mean_MAE_lr_20h = round(mean_MAE_lr_20h/len(resultats_20h),2)
mean_MAE_rf_20h = round(mean_MAE_rf_20h/len(resultats_20h),2)
mean_MAE_naif_20h = round(mean_MAE_naif_20h/len(resultats_20h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_20h, mean_std_occupation_20h, mean_MAE_lr_20h, mean_MAE_rf_20h, mean_MAE_naif_20h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 20h | Mean(Std(Occupation)) / 20h | Moyenne MAE LR / 20h | Moyenne MAE RF / 20h | Moyenne MAE méthode naïve / 20h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         17.72          |             6.02            |         7.92         |         7.08         |               8.38              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 21h

In [242]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_21h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 21h", "Std(Occupation) - 21h", "MAE LR - 21h", "MAE RF - 21h", "MAE méthode intuitive - 21h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 21h
    df_1h = sous_df_1h(df_h,21)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_21h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 21h | Std(Occupation) - 21h | MAE LR - 21h | MAE RF - 21h | MAE méthode intuitive - 21h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          6.8           |          1.79         |     4.19     |     3.85     |             4.4             |
|   station_1    |          39.4          |         16.36         |     18.4     |    16.29     |             24.0            |
|   station_2    |          3.8           |          2.39         |     2.62     |     1.71     |             1.8             |
|   station_3    |          5.6           |          4.28         |     2.93     |     2.98     |             6.8             |
|   station_4    |          23.6          |          7.3          |     8.33     |     7.85     |       

In [243]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 21h", "Mean(Std(Occupation)) / 21h", "Moyenne MAE LR / 21h", "Moyenne MAE RF / 21h", "Moyenne MAE méthode naïve / 21h"]
mean_capacity_21h = 0
mean_std_occupation_21h = 0
mean_MAE_lr_21h = 0
mean_MAE_rf_21h = 0
mean_MAE_naif_21h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_21h.items():
    mean_capacity_21h += resultats_21h[key][9]
    mean_std_occupation_21h += resultats_21h[key][15]
    mean_MAE_lr_21h += resultats_21h[key][4]
    mean_MAE_rf_21h += resultats_21h[key][6]
    mean_MAE_naif_21h += resultats_21h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_21h = round(mean_capacity_21h/len(resultats_18h),2)
mean_std_occupation_21h = round(mean_std_occupation_21h/len(resultats_18h),2)
mean_MAE_lr_21h = round(mean_MAE_lr_21h/len(resultats_18h),2)
mean_MAE_rf_21h = round(mean_MAE_rf_21h/len(resultats_18h),2)
mean_MAE_naif_21h = round(mean_MAE_naif_21h/len(resultats_18h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_21h, mean_std_occupation_21h, mean_MAE_lr_21h, mean_MAE_rf_21h, mean_MAE_naif_21h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 21h | Mean(Std(Occupation)) / 21h | Moyenne MAE LR / 21h | Moyenne MAE RF / 21h | Moyenne MAE méthode naïve / 21h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         18.04          |             5.61            |         7.86         |         6.89         |               10.0              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 22h

In [244]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_22h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 22h", "Std(Occupation) - 22h", "MAE LR - 22h", "MAE RF - 22h", "MAE méthode intuitive - 22h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 22h
    df_1h = sous_df_1h(df_h,22)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_22h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 22h | Std(Occupation) - 22h | MAE LR - 22h | MAE RF - 22h | MAE méthode intuitive - 22h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          4.75          |          3.3          |     4.67     |     4.54     |             5.75            |
|   station_1    |         42.75          |          5.56         |    15.91     |     19.1     |            24.25            |
|   station_2    |          2.25          |          1.89         |     2.03     |     1.62     |             3.0             |
|   station_3    |          8.25          |          5.56         |     3.6      |     3.52     |             5.75            |
|   station_4    |         23.75          |          6.65         |     7.3      |     7.46     |       

In [245]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 22h", "Mean(Std(Occupation)) / 22h", "Moyenne MAE LR / 22h", "Moyenne MAE RF / 22h", "Moyenne MAE méthode naïve / 22h"]
mean_capacity_22h = 0
mean_std_occupation_22h = 0
mean_MAE_lr_22h = 0
mean_MAE_rf_22h = 0
mean_MAE_naif_22h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_22h.items():
    mean_capacity_22h += resultats_22h[key][9]
    mean_std_occupation_22h += resultats_22h[key][15]
    mean_MAE_lr_22h += resultats_22h[key][4]
    mean_MAE_rf_22h += resultats_22h[key][6]
    mean_MAE_naif_22h += resultats_22h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_22h = round(mean_capacity_22h/len(resultats_18h),2)
mean_std_occupation_22h = round(mean_std_occupation_22h/len(resultats_18h),2)
mean_MAE_lr_22h = round(mean_MAE_lr_22h/len(resultats_18h),2)
mean_MAE_rf_22h = round(mean_MAE_rf_22h/len(resultats_18h),2)
mean_MAE_naif_22h = round(mean_MAE_naif_22h/len(resultats_18h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_22h, mean_std_occupation_22h, mean_MAE_lr_22h, mean_MAE_rf_22h, mean_MAE_naif_22h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 22h | Mean(Std(Occupation)) / 22h | Moyenne MAE LR / 22h | Moyenne MAE RF / 22h | Moyenne MAE méthode naïve / 22h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         18.05          |             4.31            |         8.27         |         8.68         |              11.48              |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


# Résultats à 23h

In [253]:
# Pour chaque station, on garde le numéro de la station dans le dataframe df_velib, les modèle lr et rf
resultats_23h = {}
# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Mean(Occupation) - 23h", "Std(Occupation) - 23h", "MAE LR - 23h", "MAE RF - 23h", "MAE méthode intuitive - 23h"]

# Dataframe pour la Régression Linéaire et le Random Forest
for num_station in range(len(stations_tests)):
    
    # Préparation du sous_df pour le ML pour 23h
    df_1h = sous_df_1h(df_h,23)
    df_ml = sous_df_ml(df_1h, num_station)


    # Machine Learning
    stations_features = [i for i in df_ml.columns if i != 'y']
    X = df_ml[stations_features]
    # Préparation de la target
    y = df_ml['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = False)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor(max_depth = 30, min_samples_split = 7, n_estimators = 600)
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[num_station], y_test),2)

    # Capacité moyenne de la station sur la période observée
    occupation_moyenne = round(X_test[num_station].mean(),2)
    # Variance de la capacité sur la période observée
    variance_occupation = round(X_test[num_station].std(),2)

    # Mise en forme des résultats
    y_test = pd.DataFrame(y_test)
    index = y_test.index
    # Renommer pour plus de clarté
    y_prediction_lr = pd.DataFrame(y_prediction_lr, index = index)
    y_prediction_rf = pd.DataFrame(y_prediction_rf, index = index)
    y_naif = pd.DataFrame(X_test[num_station])
    y_prediction_naif = y_test

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_23h[stations_tests[num_station]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, occupation_moyenne, y_test, y_prediction_lr, y_prediction_rf, y_naif, y_prediction_naif, variance_occupation]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[num_station], occupation_moyenne, variance_occupation, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
| Station target | Mean(Occupation) - 23h | Std(Occupation) - 23h | MAE LR - 23h | MAE RF - 23h | MAE méthode intuitive - 23h |
+----------------+------------------------+-----------------------+--------------+--------------+-----------------------------+
|   station_0    |          5.4           |          3.21         |     4.43     |     4.02     |             4.8             |
|   station_1    |          44.8          |          6.76         |    15.45     |    16.01     |             22.2            |
|   station_2    |          3.8           |          3.03         |     1.57     |     1.25     |             1.4             |
|   station_3    |          6.4           |          4.88         |     1.42     |     2.17     |             5.2             |
|   station_4    |          24.6          |          5.59         |     7.07     |     6.68     |       

In [254]:
ptbl = PrettyTable()
ptbl.field_names = ["Mean(Occupation) / 23h", "Mean(Std(Occupation)) / 23h", "Moyenne MAE LR / 23h", "Moyenne MAE RF / 23h", "Moyenne MAE méthode naïve / 23h"]
mean_capacity_23h = 0
mean_std_occupation_23h = 0
mean_MAE_lr_23h = 0
mean_MAE_rf_23h = 0
mean_MAE_naif_23h = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_23h.items():
    mean_capacity_23h += resultats_23h[key][9]
    mean_std_occupation_23h += resultats_23h[key][15]
    mean_MAE_lr_23h += resultats_23h[key][4]
    mean_MAE_rf_23h += resultats_23h[key][6]
    mean_MAE_naif_23h += resultats_23h[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_23h = round(mean_capacity_23h/len(resultats_23h),2)
mean_std_occupation_23h = round(mean_std_occupation_23h/len(resultats_23h),2)
mean_MAE_lr_23h = round(mean_MAE_lr_23h/len(resultats_23h),2)
mean_MAE_rf_23h = round(mean_MAE_rf_23h/len(resultats_23h),2)
mean_MAE_naif_23h = round(mean_MAE_naif_23h/len(resultats_23h),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_23h, mean_std_occupation_23h, mean_MAE_lr_23h, mean_MAE_rf_23h, mean_MAE_naif_23h])
print(ptbl)

+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
| Mean(Occupation) / 23h | Mean(Std(Occupation)) / 23h | Moyenne MAE LR / 23h | Moyenne MAE RF / 23h | Moyenne MAE méthode naïve / 23h |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+
|         18.88          |             4.57            |         7.32         |         7.01         |               9.6               |
+------------------------+-----------------------------+----------------------+----------------------+---------------------------------+


In [248]:
heures

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [255]:
erreur_pred_lr_h_par_h = [mean_MAE_lr_0h, mean_MAE_lr_1h, mean_MAE_lr_2h, mean_MAE_lr_3h, mean_MAE_lr_4h, mean_MAE_lr_5h, mean_MAE_lr_6h, mean_MAE_lr_7h, mean_MAE_lr_8h, mean_MAE_lr_9h, mean_MAE_lr_10h, mean_MAE_lr_11h, mean_MAE_lr_12h, mean_MAE_lr_13h, mean_MAE_lr_14h, mean_MAE_lr_15h, mean_MAE_lr_16h, mean_MAE_lr_17h, mean_MAE_lr_18h, mean_MAE_lr_19h, mean_MAE_lr_20h, mean_MAE_lr_21h, mean_MAE_lr_22h, mean_MAE_lr_23h]

erreur_pred_rf_h_par_h = [mean_MAE_rf_0h, mean_MAE_rf_1h, mean_MAE_rf_2h, mean_MAE_rf_3h, mean_MAE_rf_4h, mean_MAE_rf_5h, mean_MAE_rf_6h, mean_MAE_rf_7h, mean_MAE_rf_8h, mean_MAE_rf_9h, mean_MAE_rf_10h, mean_MAE_rf_11h, mean_MAE_rf_12h, mean_MAE_rf_13h, mean_MAE_rf_14h, mean_MAE_rf_15h, mean_MAE_rf_16h, mean_MAE_rf_17h, mean_MAE_rf_18h, mean_MAE_rf_19h, mean_MAE_rf_20h, mean_MAE_rf_21h, mean_MAE_rf_22h, mean_MAE_rf_23h]

erreur_pred_naif_h_par_h = [mean_MAE_naif_0h, mean_MAE_naif_1h, mean_MAE_naif_2h, mean_MAE_naif_3h, mean_MAE_naif_4h, mean_MAE_naif_5h, mean_MAE_naif_6h, mean_MAE_naif_7h, mean_MAE_naif_8h, mean_MAE_naif_9h, mean_MAE_naif_10h, mean_MAE_naif_11h, mean_MAE_naif_12h, mean_MAE_naif_13h, mean_MAE_naif_14h, mean_MAE_naif_15h, mean_MAE_naif_16h, mean_MAE_naif_17h, mean_MAE_naif_18h, mean_MAE_naif_19h, mean_MAE_naif_20h, mean_MAE_naif_21h, mean_MAE_naif_22h, mean_MAE_naif_23h]

In [263]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_lr'],
    name="Erreur moyenne avec Régression Linéaire",
    line=dict(
        color='#636EFA'
    )      
))

fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_lr_h_par_h,
    name="Erreur moyenne avec Régression Linéaire / 1 modèle par heure",
    line=dict(
        color='#636EFA',
        width=1,
        dash="dot"
    )     
))

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_rf'],
    name="Erreur moyenne avec Forêt aléatoire",
    line=dict(
        color='#EF553B'
    )     
))

fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_rf_h_par_h,
    name="Erreur moyenne avec Forêt aléatoire / 1 modèle par heure",
    line=dict(
        color='#EF553B',
        width=1,
        dash="dot"
    )     
))

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_naif'],
    name="Erreur moyenne avec la méthode naïve",
    line=dict(
        color='#00CC96'
    )     
))

fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_naif_h_par_h,
    name="Erreur moyenne avec la méthode naïve / 1 modèle par heure",
    line=dict(
        color='#00CC96',
        width=1,
        dash="dot"
    )     
))

# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[4,4, 12.5,12.5],
    y=[11,10.5, 11,10.5],
    text=["Nuit", "19h-5h59",
          "Journée", "6h-18h59"],
    mode="text",
    showlegend = False

))

# Set axes ranges
fig.update_xaxes(range=[0, 23])
fig.update_yaxes(range=[5, 12.5])

# Add shapes
fig.add_shape(type="line",
    x0=6, y0=0, x1=6, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=19, y0=0, x1=19, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)




fig.update_layout(
    title="Erreurs de prédictions sur les 10 stations du 04/01/21 au 12/02/21",
    xaxis_title="Heure",
    yaxis_title="Erreur moyenne",
    legend_title="",
    autosize=False,
    width=1000,
    height=500
)

fig.show()

In [261]:
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_lr_h_par_h,
    name="Erreur moyenne avec Régression Linéaire / 1 modèle par heure",
    line=dict(
        color='#636EFA',
        width=2,
        dash="dot"
    )     
))


fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_rf_h_par_h,
    name="Erreur moyenne avec Forêt aléatoire / 1 modèle par heure",
    line=dict(
        color='#EF553B',
        width=2,
        dash="dot"
    )     
))


fig.add_trace(go.Scatter(
    x=heures,
    y=erreur_pred_naif_h_par_h,
    name="Erreur moyenne avec la méthode naïve / 1 modèle par heure",
    line=dict(
        color='#00CC96',
        width=2,
        dash="dot"
    )     
))

# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[4,4, 12.5,12.5],
    y=[11,10.5, 11,10.5],
    text=["Nuit", "19h-5h59",
          "Journée", "6h-18h59"],
    mode="text",
    showlegend = False

))

# Set axes ranges
fig.update_xaxes(range=[0, 23])
fig.update_yaxes(range=[5, 12.5])

# Add shapes
fig.add_shape(type="line",
    x0=6, y0=0, x1=6, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=19, y0=0, x1=19, y1=12.5,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)




fig.update_layout(
    title="Erreurs de prédictions sur les 10 stations du 04/01/21 au 12/02/21",
    xaxis_title="Heure",
    yaxis_title="Erreur moyenne",
    legend_title="",
    autosize=False,
    width=1000,
    height=500
)

fig.show()

## La nuit : 22h-5h

In [30]:
# On identifie les heures de nuit dans notre dataframe
heures_non_nuit = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
test = df_h.isin({'heure': heures_non_nuit})

for i in df_h.index:
    if test['heure'][i] == True:
        df_h.drop(i,0,inplace=True)

df_h

,0,1,2,3,4,5,6,7,8,9,...,1389,1390,1391,1392,1393,1394,1395,1396,1397,heure
10/19/20;22:57:14,3,52,11,10,23,24,30,7,14,36,...,13,0,18,19,32,16,35,12,7,22
10/19/20;23:57:18,3,52,9,10,24,23,31,7,14,36,...,12,0,18,19,33,16,35,12,7,23
10/20/20;00:57:22,3,52,9,11,24,23,31,7,14,36,...,12,0,17,19,33,16,35,12,7,0
10/20/20;01:57:26,3,52,9,11,24,22,31,7,14,37,...,12,1,17,19,32,16,35,12,7,1
10/20/20;02:57:30,2,52,9,11,24,22,31,7,13,37,...,12,1,17,19,32,16,35,12,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02/15/21;01:14:44,6,38,3,4,23,39,7,12,33,52,...,16,2,10,9,32,5,20,12,11,1
02/15/21;02:18:15,6,38,3,4,23,26,7,12,34,52,...,16,2,10,9,33,5,20,12,11,2
02/15/21;03:21:43,6,39,3,5,23,26,7,12,34,51,...,16,2,10,9,33,5,20,12,11,3
02/15/21;04:25:14,6,39,3,5,23,26,7,12,34,52,...,16,2,10,9,33,5,20,12,11,4


On voit bien qu'on dispose de 924 données correspondant aux heures de nuit, ce qui est cohérent avec la longueur initiale du dataframe de 2778 données.

In [32]:
# Stockage des index du dataframe
axis = df_h.index
# Préparation des résultats à stocker : pour chaque station, on garde le numéro de la station dans le dataframe df_velib, le modèle de 
# régression linéaire et de random forest
resultats_nuit = {}

# Préparation de la visualisation des résultats :
ptbl = PrettyTable()
ptbl.field_names = ["Station target", "Capacité", "Occupation moyenne / nuit", "MAE LR / nuit", "MAE RF / nuit", "MAE méthode naïve /nuit"]


for i in range(len(stations_tests)):
    df = df_h.copy()
    df['y'] = df[i]

    for j in range(len(df)-24):
        df['y'][axis[j]] = df[i][axis[j+24]]
        
    # Puis on supprime les 24 dernières lignes qui ne peuvent pas être complétées
    df.drop(axis[-24:], inplace = True)

    # Machine Learning
    stations_features = [i for i in df.columns if type(i) != str]
    X = df[stations_features]
    # Préparation de la target
    y = df['y']
    # Séparation en données de test et d'entrainement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # 1- Régression linéaire
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_prediction_lr = lr.predict(X_test)
    MAE_lr = round(mean_absolute_error(y_test, y_prediction_lr),2)

    # 2- Random Forest
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_prediction_rf = rf.predict(X_test)
    MAE_rf = round(mean_absolute_error(y_test, y_prediction_rf),2)

    # 3- Méthode naïve ou bien "intuitive" c'est-à-dire prédire que ce qui se passe 1 jour plus tard est exactement la même chose que la            veille
    df_naif = X_test.copy()
    #MAE_naif = round(abs(X_test[i]-y_test).mean() , 2)
    MAE_naif = round(mean_absolute_error(X_test[i], y_test),2)

    # Capacité moyenne de la station sur la période observée
    capacite_moyenne = round(df[i].mean(),2)

    # Stockage des résultats : numéro de la station, les modèles, et les MAE de chaque modèle
    resultats_nuit[stations_tests[i]] = [i, X, y, lr, MAE_lr, rf, MAE_rf, df_naif, MAE_naif, capacite_moyenne]

    # Visualisation des performances avec Prettytable
    ptbl.add_row([stations_tests[i], velib_details['capacity'][i], capacite_moyenne, MAE_lr, MAE_rf, MAE_naif])
    
print(ptbl)

+----------------+----------+---------------------------+---------------+---------------+-------------------------+
| Station target | Capacité | Occupation moyenne / nuit | MAE LR / nuit | MAE RF / nuit | MAE méthode naïve /nuit |
+----------------+----------+---------------------------+---------------+---------------+-------------------------+
|   station_0    |    35    |            5.69           |      1.66     |      0.85     |           3.6           |
|   station_1    |    55    |           29.97           |      3.28     |      2.5      |          12.67          |
|   station_2    |    20    |            5.73           |      1.64     |      1.02     |           3.67          |
|   station_3    |    21    |            7.22           |      2.25     |      1.3      |           4.1           |
|   station_4    |    30    |           21.32           |      2.76     |      1.86     |           6.89          |
|   station_5    |    46    |           27.87           |      2.96     

In [33]:
ptbl = PrettyTable()
ptbl.field_names = ["Occupation moyenne / nuit", "Moyenne MAE LR / nuit", "Moyenne MAE RF / nuit", "Moyenne MAE méthode naïve / nuit"]
mean_capacity_nuit = 0
mean_MAE_lr_nuit = 0
mean_MAE_rf_nuit = 0
mean_MAE_naif_nuit = 0

# Parcourir le dictionnaire des résultats pour calculer la moyenne des indicateurs
for key, item in resultats_nuit.items():
    mean_capacity_nuit += resultats_nuit[key][9]
    mean_MAE_lr_nuit += resultats_nuit[key][4]
    mean_MAE_rf_nuit += resultats_nuit[key][6]
    mean_MAE_naif_nuit += resultats_nuit[key][8]

# On divise par le nombre total de stations tests pour obtenir la moyenne
mean_capacity_nuit = round(mean_capacity_nuit/len(resultats_nuit),2)
mean_MAE_lr_nuit = round(mean_MAE_lr_nuit/len(resultats_nuit),2)
mean_MAE_rf_nuit = round(mean_MAE_rf_nuit/len(resultats_nuit),2)
mean_MAE_naif_nuit = round(mean_MAE_naif_nuit/len(resultats_nuit),2)

# Visualisation des performances avec Prettytable
ptbl.add_row([mean_capacity_nuit, mean_MAE_lr_nuit, mean_MAE_rf_nuit, mean_MAE_naif_nuit])
print(ptbl)

+---------------------------+-----------------------+-----------------------+----------------------------------+
| Occupation moyenne / nuit | Moyenne MAE LR / nuit | Moyenne MAE RF / nuit | Moyenne MAE méthode naïve / nuit |
+---------------------------+-----------------------+-----------------------+----------------------------------+
|           17.24           |          2.47         |          1.7          |               7.27               |
+---------------------------+-----------------------+-----------------------+----------------------------------+


On regarde ensuite si les prédictions séparées conduisent à de meilleurs résultats par rapport à un modèle général

In [47]:
mean_MAE_lr_decoupage = round((mean_MAE_lr_nuit * 8 + mean_MAE_lr_matin * 3 + mean_MAE_lr_heures_creuses * 7 + mean_MAE_lr_fin_travail * 4 + mean_MAE_lr_soir * 2)/24,2)

mean_MAE_rf_decoupage = round((mean_MAE_rf_nuit * 8 + mean_MAE_rf_matin * 3 + mean_MAE_rf_heures_creuses * 7 + mean_MAE_rf_fin_travail * 4 + mean_MAE_rf_soir * 2)/24,2)

mean_MAE_naif_decoupage = round((mean_MAE_naif_nuit * 8 + mean_MAE_naif_matin * 3 + mean_MAE_naif_heures_creuses * 7 + mean_MAE_naif_fin_travail * 4 + mean_MAE_naif_soir * 2)/24,2)

In [48]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_lr'],
    name="Erreur moyenne avec Régression Linéaire"      
))


fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_rf'],
    name="Erreur moyenne avec Forêt aléatoire"     
))

fig.add_trace(go.Scatter(
    x=mean_erreur_par_heure.index,
    y=mean_erreur_par_heure['erreur_pred_naif'],
    name="Erreur moyenne avec la méthode naïve"     
))


# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[3,3, 7,7, 12.5,12.5, 17.5,17.5, 20,20],
    y=[4,3.75, 1,0.75, 1,0.75, 1,0.75, 1,0.75],
    text=["Nuit", "22h-5h",
          "Matin", "6h-8h",
          "Les heures creuses", "9h-16h",
          "Fin travail", "17h-18h",
          "Soir", "19h-21h"],
    mode="text",
    showlegend = False

))

# Set axes ranges
fig.update_xaxes(range=[0, 23])
fig.update_yaxes(range=[0, 7])

# Add shapes
fig.add_shape(type="line",
    x0=5.5, y0=0, x1=5.5, y1=7,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=8.5, y0=0, x1=8.5, y1=7,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=16.5, y0=0, x1=16.5, y1=7,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=18.5, y0=0, x1=18.5, y1=7,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)

fig.add_shape(type="line",
    x0=21.5, y0=0, x1=21.5, y1=7,
    line=dict(
        color="black",
        width=2,
        dash="dot"
    )
)



fig.update_layout(
    title="Choix des intervalles de temps à prédire au regard des 10 stations",
    xaxis_title="Heure",
    yaxis_title="Erreur moyenne",
    legend_title="",
    autosize=False,
    width=1000,
    height=500
)

fig.show()

In [49]:
ptbl5 = PrettyTable()
ptbl5.field_names = ["Occupation moyenne - test", "Moyenne MAE LR - 5 modèles - test", "Moyenne MAE RF - 5 modèles - test", "Moyenne MAE méthode naïve - 5 modèles - test"]

# Visualisation des performances avec Prettytable
ptbl5.add_row([mean_occupation1, mean_MAE_lr_decoupage, mean_MAE_rf_decoupage, mean_MAE_naif_decoupage])
print(ptbl5)

+---------------------------+-----------------------------------+-----------------------------------+----------------------------------------------+
| Occupation moyenne - test | Moyenne MAE LR - 5 modèles - test | Moyenne MAE RF - 5 modèles - test | Moyenne MAE méthode naïve - 5 modèles - test |
+---------------------------+-----------------------------------+-----------------------------------+----------------------------------------------+
|           15.84           |                3.28               |                3.12               |                     7.36                     |
+---------------------------+-----------------------------------+-----------------------------------+----------------------------------------------+


In [50]:
print(ptbl1)

+---------------------------+----------------------------------+----------------------------------+---------------------------------------------+
| Occupation moyenne - test | Moyenne MAE LR - 1 modèle - test | Moyenne MAE RF - 1 modèle - test | Moyenne MAE méthode naïve - 1 modèle - test |
+---------------------------+----------------------------------+----------------------------------+---------------------------------------------+
|           15.84           |               3.65               |               2.34               |                     5.59                    |
+---------------------------+----------------------------------+----------------------------------+---------------------------------------------+
